In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Subset

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
import os
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from pathlib import Path
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [5]:
import importlib
try: 
    importlib.reload(utils)
    importlib.reload(generators)
    importlib.reload(trainers)
    print("libs reloaded")
except:
    import utils  
    import generators 
    import trainers
    print("libs imported")
  
#Prostate2D = utils.Prostate2D
VAE = generators.VAE
VAEGAN = generators.VAEGAN
TrainerVAE = trainers.TrainerVAE
TrainerVAEGAN = trainers.TrainerVAEGAN

libs reloaded


In [6]:
WORKING_DIR = Path(r"C:\Users\marti\OneDrive - TU Eindhoven\Documenten\Master\Q3\Capita Selecta\Project")
DATA_DIR = WORKING_DIR / "Data"
PROGRESS_DIR_VAE = WORKING_DIR / "progress_vae"
PROGRESS_DIR_VAE.mkdir(parents=True, exist_ok=True)

PROGRESS_DIR_VAEGAN = WORKING_DIR / "progress_vaegan"
PROGRESS_DIR_VAEGAN.mkdir(parents=True, exist_ok=True)

Z_DIM = 256
BATCH_SIZE = 32
N_EPOCHS = 200
DECAY_LR_AFTER = 150 #50
LEARNING_RATE = 1e-3
OPTIMIZER = lambda parameters, lr : torch.optim.Adam(parameters, lr=lr, betas=(0.,0.9))
KLD_ANNEALING_EPOCHS = 50
GAMMA = 1.0
ADA_TARGET = 0.6
ADA_LENGTH = 10_000
ACCUM = 0.5 ** (32 / (10 * 1000))
NO_VALIDATION_PATIENTS = 2
IMAGE_SIZE = (64,64)
TOLERANCE = 0.01
MINIMUM_VALID_LOSS = 10 

exp = str(N_EPOCHS) + '_epochs_' + str(Z_DIM) + "_zdim"
EXPERIMENT_DIR_VAE = PROGRESS_DIR_VAE / exp
EXPERIMENT_DIR_VAE.mkdir(parents=True, exist_ok=True)
EXPERIMENT_DIR_VAEGAN = PROGRESS_DIR_VAEGAN / exp
EXPERIMENT_DIR_VAEGAN.mkdir(parents=True, exist_ok=True)

def lambda_lr(the_epoch):
    return (
        1.0
        if the_epoch < DECAY_LR_AFTER
        else 1 - float(the_epoch - DECAY_LR_AFTER) / (N_EPOCHS - DECAY_LR_AFTER)
    )

patients = [
    path
    for path in DATA_DIR.glob("*")
    if not any(part.startswith(".") for part in path.parts)
]

#train_indx, valid_indx = train_test_split(patients, random_state=seed, train_size=train_size)

random.shuffle(patients)

# split in training/validation after shuffling
partition = {
    "train": patients[:-NO_VALIDATION_PATIENTS],
    "validation": patients[-NO_VALIDATION_PATIENTS:],
}

# load training data and create DataLoader with batching and shuffling
train_set = utils.ProstateMRDataset(partition["train"], IMAGE_SIZE)
train_loader = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
)

# load validation data
valid_set = utils.ProstateMRDataset(partition["validation"], IMAGE_SIZE)
valid_loader = DataLoader(
    valid_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
)

In [7]:
vae_model = VAE(z_dim=Z_DIM, spade=True).to(device)
optimizer = OPTIMIZER(vae_model.parameters(), lr=LEARNING_RATE)
vae_trainer = TrainerVAE(
    vae_model,
    optimizer,
    KLD_ANNEALING_EPOCHS,
    EXPERIMENT_DIR_VAE,
    train_loader,
    valid_loader,
    EXPERIMENT_DIR_VAE,
    TOLERANCE,
    MINIMUM_VALID_LOSS,
    device,
    seed = 0,
)
     

In [8]:
vae_trainer.train(N_EPOCHS, lambda_lr=lambda_lr)

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch #000: Rec_Loss/train = 0.331, KLD/train = 1.636 | Rec_Loss/valid = 0.261, KLD/valid = 2.239


  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch #010: Rec_Loss/train = 0.162, KLD/train = 0.336 | Rec_Loss/valid = 0.231, KLD/valid = 0.206


  0%|          | 0/34 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plt.plot(range(1,N_EPOCHS+1), vae_trainer.train_losses)
plt.plot(range(1,N_EPOCHS+1), vae_trainer.valid_losses)
plt.xlabel('Number of epochs')
plt.ylabel('Reconstruction loss')
plt.title('Reconstruction loss mask vae')
plt.legend(['Training set', 'Validation set'])
plot_name = 'RECON_LOSS_'+str(N_EPOCHS) + '_epochs_' + str(Z_DIM) + '_zdim.png'
path = EXPERIMENT_DIR_VAEGAN / plot_name

plt.savefig(path, dpi=200)

In [ ]:
OPTIMIZER = lambda parameters, lr : torch.optim.RMSprop(
    parameters,
    lr=lr,
    alpha=0.9,
    eps=1e-8,
    weight_decay=0,
    momentum=0,
    centered=False
)

vaegan_model = VAEGAN(z_dim=Z_DIM, l=2, spade=True).to(device)
net_ema = VAEGAN(z_dim=Z_DIM, l=2, spade=True).to(device)
net_ema.eval()

optimizer_enc = OPTIMIZER(vaegan_model.encoder.parameters(), lr=LEARNING_RATE)
optimizer_gen = OPTIMIZER(vaegan_model.generator.parameters(), lr=LEARNING_RATE)
optimizer_disc = OPTIMIZER(vaegan_model.discriminator.parameters(), lr=LEARNING_RATE)

vaegan_trainer = TrainerVAEGAN(
    vaegan_model,
    optimizer_enc,
    optimizer_gen,
    optimizer_disc,
    KLD_ANNEALING_EPOCHS,
    EXPERIMENT_DIR_VAEGAN,
    train_loader,
    valid_loader,
    EXPERIMENT_DIR_VAEGAN,
    TOLERANCE, 
    MINIMUM_VALID_LOSS,
    net_ema = net_ema,
    accum = ACCUM,
    ada_target = ADA_TARGET,
    ada_length = ADA_LENGTH,
    gamma = GAMMA,
    device = device,
    seed = 0,
)

cuda


In [ ]:
vaegan_trainer.train(N_EPOCHS, lambda_lr=lambda_lr)

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch #000: Rec_Loss/train = 0.600, KLD/train = 9.670, Discl_Loss/train = 1.622, Adv_Loss/train = 1.151 | Rec_Loss/valid = 0.421, KLD/valid = 7.849, Discl_Loss/valid = 1.396, Adv_Loss/valid = 0.051


  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch #198: Rec_Loss/train = 0.297, KLD/train = 0.080, Discl_Loss/train = 0.640, Adv_Loss/train = 0.404 | Rec_Loss/valid = 0.260, KLD/valid = 0.048, Discl_Loss/valid = 1.102, Adv_Loss/valid = 1.747


  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch #199: Rec_Loss/train = 0.298, KLD/train = 0.080, Discl_Loss/train = 0.639, Adv_Loss/train = 0.391 | Rec_Loss/valid = 0.260, KLD/valid = 0.048, Discl_Loss/valid = 1.095, Adv_Loss/valid = 1.724


In [ ]:
plt.plot(range(1,N_EPOCHS+1), vaegan_trainer.ada_p_log)
plt.plot(range(1,N_EPOCHS+1), vaegan_trainer.valid_losses)
plt.xlabel('Number of epochs')
plt.ylabel('Reconstruction loss')
plt.title('Reconstruction loss mask vae')
plt.legend(['Training set', 'Validation set'])
plot_name = 'RECON_LOSS_'+str(N_EPOCHS) + '_epochs_' + str(Z_DIM) + '_zdim.png'
path = EXPERIMENT_DIR_VAEGAN / plot_name

plt.savefig(path, dpi=200)